In [ ]:
import cv2
import numpy as np
import tensorflow as tf

### 25200개의 바운딩박스에서 좋은 바운딩박스만 걸러내고,

### Non-Maximun Suppresion으로 겹쳐진 바운딩박스 정리하기

1. 앞 단계와 동일하게 모델 로드 후 테스트 이미지를 넣고 추론을 실시합니다.

In [ ]:
model = tf.saved_model.load("../best_saved_model")
src = cv2.imread("../test_images/test_image.png")
src_converted = cv2.cvtColor(src, cv2.COLOR_BGR2RGB)
src_resized = cv2.resize(src_converted, (640, 640))
src_resized = src_resized.reshape((1, 640, 640, 3))
src_resized = src_resized/255.
pred = model(src_resized)
pred = pred[0][0].numpy() # -> 추론결과를 담은 (25200, 7) 모양의 numpy ndarray

2. 추론 결과를 obj conf.로 cutoff하여(필터링) 후보군을 추려내겠습니다.

In [ ]:
conf_thred = 0.25 # obj conf.값으로 cutoff할 기준
class_ids = [] #검출한 바운딩박스 결과들에서 클래스 아이디만 뽑아 담아놓을 리스트
confidences = [] #검출한 바운딩박스 결과들에서 conf.만 뽑아 담아놓을 리스트
boxes = [] #검출한 바운딩박스 결과들에서 바운딩박스 좌표만 뽑아 담아놓을 리스트
for detection in pred: #detection -> [cx, cy, w, h, obj conf, cls1 conf, cls2 conf]
    class_scores = detection[5:]
    class_id = np.argmax(detection[5:]) #5번째 이후(6~)부터가 클래스에 대한 확률값을 담고 있는 배열
    confidence = detection[4] #4번째 값이 obj conf.
    if confidence >= conf_thred: #obj conf.가 conf_thred 값 이상일 경우에만 append
        cx = detection[0] #바운딩박스의 center x
        cy = detection[1] #바운딩박스의 center y
        w = detection[2] #바운딩박스의 center width
        h = detection[3] #바운딩박스의 center height
        x = cx - (w/2) #바운딩박스의 center x를 좌상단 x 좌표로 변환
        y = cy - (h/2) #바운딩박스의 center y를 좌상단 y 좌표로 변환
        
        class_ids.append(class_id) #클래스 번호를 class_ids 결과 리스트에 append
        confidences.append(confidence) #obj conf.값을 confidences 결과 리스트에 append
        boxes.append([x, y, w, h]) #바운딩 박스 정보를 boxes 결과 리스트에 append
len(boxes) # -> 25200개 중 102개만 필터링되어 출력

3. 추려낸 후보군의 바운딩박스를 이미지로 그려 확인해 보겠습니다.

In [ ]:
class_names = ["sign", "light"]
bbox_colors = [(0, 255, 0), (0, 0, 255)] #표지판은 초록색으로, 신호등은 붉은색으로 표현

#이미지가 크므로 리사이즈
dst = cv2.resize(src, (int((900*src.shape[1])/src.shape[0]), 900), interpolation=cv2.INTER_LINEAR)
for idx in range(len(boxes)): #boxes의 길이만큼 반복하면서 바운딩박스를 생성
    class_id = class_ids[idx] #class_ids에 현재 접근중인 idx로 인덱싱하여 class_id를 뽑아옴
    class_name = class_names[class_id] #뽑아온 class_id로 class_names에 접근해 class_name을 지정
    color = bbox_colors[class_id] #뽑아온 class_id로 colors에 접근해 바운딩박스 색을 지정
    bbox = boxes[idx] #boxes에 현재 접근중인 idx로 인덱싱하여 바운딩박스 정보를 가져옴
    confidence = confidences[idx] #confidences에 현재 접근중인 idx로 인덱싱하여 obj conf. 값을 가져옴

    #dst의 실사이즈에 맞춰 바운딩박스 좌표정보를 절대값으로 변환
    x = int(bbox[0] * dst.shape[1])
    y = int(bbox[1] * dst.shape[0])
    w = int(bbox[2] * dst.shape[1])
    h = int(bbox[3] * dst.shape[0])

    #바운딩박스를 그리고 class_name을 putText로 출력
    dst = cv2.rectangle(dst, (x, y, w, h), color, 2)
    dst = cv2.putText(dst, "{}:{:.2f}".format(class_name, confidence), (x, y-5), cv2.FONT_HERSHEY_DUPLEX, 0.6, color, 1)
cv2.imshow("dst", dst)
cv2.waitKey()
cv2.destroyAllWindows()

확률값이 0.25보다 낮은 바운딩박스는 모두 제거되어 이전보다 매우 깔끔해진 결과를 볼 수 있습니다. 하지만, 여전히 하나의 오브젝트에 너무 많은 바운딩박스가 겹쳐진 채로 출력되고 있습니다.

4. Non Maximum Suppression으로 겹쳐진 바운딩박스들을 하나로 정리하겠습니다.

In [ ]:
nms_thres = 0.45 #nms에서 사용하는 IoU의 threshold 값을 설정

"""
opencv의 NMS 함수를 활용해 비최대억제 실시
필요시 사용자 함수로 구현가능하며, 부록 코드를 확인할 것
"""
boxes_with_nms = cv2.dnn.NMSBoxes(boxes, confidences, conf_thred, nms_thres) 

#실제 눈으로 확인
dst = cv2.resize(src, (int((900*src.shape[1])/src.shape[0]), 900), interpolation=cv2.INTER_LINEAR)
for idx in boxes_with_nms:
    class_id = class_ids[idx]
    class_name = class_names[class_id]
    color = bbox_colors[class_id]
    bbox = boxes[idx]
    confidence = confidences[idx]

    x = int(bbox[0] * dst.shape[1])
    y = int(bbox[1] * dst.shape[0])
    w = int(bbox[2] * dst.shape[1])
    h = int(bbox[3] * dst.shape[0])

    dst = cv2.rectangle(dst, (x, y, w, h), color, 2)
    dst = cv2.putText(dst, "{}:{:.2f}".format(class_name, confidence), (x, y-5), cv2.FONT_HERSHEY_DUPLEX, 0.6, color, 1)
cv2.imshow("dst", dst)
cv2.waitKey()
cv2.destroyAllWindows()

겹쳐진 바운딩박스가 정리되어 결과가 훨씬 깔끔하게 출력되었습니다.

다른 클래스의 바운딩박스가 서로 겹쳐져 있을 때는 어떻게 할 것인지, 작은 바운딩박스가 여러개 겹쳐져 있을 때 Merging을 어떻게 할 것인지 등 다양한 케이스가 존재할텐데 이러한 추가적인 후처리는 학습목적상 생략하였습니다. 부록 코드를 참고해 추후 스스로 구현해 보기 바랍니다.

### 동영상에 적용해 결과 보기

In [ ]:
conf_thres = 0.25
nms_thres = 0.45
class_names = ["sign", "light"]
bbox_colors = [(0, 255, 0), (0, 0, 255)]

# cap = cv2.VideoCapture(0) #카메라로 열 경우
cap = cv2.VideoCapture("../test_images/test_video.mp4")

while True:
    retval, frame = cap.read()
    if retval is False:
        break

    src = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    src = cv2.resize(src, (640, 640))
    src = src/255.
    src = src.reshape((1, 640, 640, 3))
    pred = model(src)
    pred = pred[0][0].numpy()

    class_ids = [] #검출한 바운딩박스 결과들에서 클래스 아이디만 뽑아 담아놓을 리스트
    confidences = [] #검출한 바운딩박스 결과들에서 conf.만 뽑아 담아놓을 리스트
    boxes = [] #검출한 바운딩박스 결과들에서 바운딩박스 좌표만 뽑아 담아놓을 리스트
    for detection in pred: #detection -> [cx, cy, w, h, obj conf, cls1 conf, cls2 conf]
        class_scores = detection[5:]
        class_id = np.argmax(detection[5:]) #5번째 이후(6~)부터가 클래스에 대한 확률값을 담고 있는 배열
        confidence = detection[4] #4번째 값이 obj conf.
        if confidence >= conf_thred: #obj conf.가 conf_thred 값 이상일 경우에만 append
            cx = detection[0] #바운딩박스의 center x
            cy = detection[1] #바운딩박스의 center y
            w = detection[2] #바운딩박스의 center width
            h = detection[3] #바운딩박스의 center height
            x = cx - (w/2) #바운딩박스의 center x를 좌상단 x 좌표로 변환
            y = cy - (h/2) #바운딩박스의 center y를 좌상단 y 좌표로 변환
            
            class_ids.append(class_id) #클래스 번호를 class_ids 결과 리스트에 append
            confidences.append(confidence) #obj conf.값을 confidences 결과 리스트에 append
            boxes.append([x, y, w, h]) #바운딩 박스 정보를 boxes 결과 리스트에 append

    boxes_with_nms = cv2.dnn.NMSBoxes(boxes, confidences, conf_thred, nms_thres) 

    dst = cv2.resize(frame, (int((900*frame.shape[1])/frame.shape[0]), 900), interpolation=cv2.INTER_LINEAR)
    for idx in boxes_with_nms:
        class_id = class_ids[idx]
        class_name = class_names[class_id]
        color = bbox_colors[class_id]
        bbox = boxes[idx]
        confidence = confidences[idx]

        x = int(bbox[0] * dst.shape[1])
        y = int(bbox[1] * dst.shape[0])
        w = int(bbox[2] * dst.shape[1])
        h = int(bbox[3] * dst.shape[0])

        dst = cv2.rectangle(dst, (x, y, w, h), color, 2)
        dst = cv2.putText(dst, "{}:{:.2f}".format(class_name, confidence), (x, y-5), cv2.FONT_HERSHEY_DUPLEX, 0.6, color, 1)
    cv2.imshow("dst", dst)
    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()
cap.release()

### Lessons

1. 물체감지는 라벨링 작업에 시간이 많이 걸리므로 이번 실습은 일반에 공개된 데이터를 활용해 진행할 수 밖에 없었습니다. 현업에서 실제 활용할 목적으로 모델을 만들고자 한다면, 이미지 수집과 함께 라벨링 작업을 진행해 주어야 합니다.

2. 모델을 만드는 것 보다 데이터 전처리/후처리가 더 힘들고 공력이 많이 들어갑니다. 알고리즘에 대한 이해도 중요하지만, 전처리/후처리를 위한 코딩 기본기도 잘 닦아 놓기 바랍니다.